In [1]:
import pandas as pd
import numpy as np
import random
from math import log

In [2]:
# read excel and geenrate dataframe
np.random.seed(3)
raw_dataset = pd.DataFrame(pd.read_excel(r'dataset_watermelon3.xlsx'))
raw_dataset = raw_dataset.sample(frac = 1).reset_index(drop=True) # shuffle
#raw_dataset.loc[raw_dataset['好瓜']=='是','好瓜']=1
#raw_dataset.loc[raw_dataset['好瓜']=='否','好瓜']=0
print(raw_dataset)
train_set = raw_dataset.loc[:int(0.7*len(raw_dataset.index))-1]
test_set = raw_dataset.loc[int(0.7*len(raw_dataset.index)):]
print(len(raw_dataset),len(train_set),len(test_set))

    色泽  根蒂  敲声  纹理  脐部  触感     密度    含糖率 好瓜
0   青绿  蜷缩  沉闷  稍糊  稍凹  硬滑  0.719  0.103  否
1   乌黑  稍蜷  浊响  稍糊  稍凹  软粘  0.481  0.149  是
2   浅白  蜷缩  浊响  清晰  凹陷  硬滑  0.556  0.215  是
3   乌黑  稍蜷  浊响  清晰  稍凹  硬滑  0.437  0.211  是
4   乌黑  稍蜷  浊响  清晰  稍凹  软粘  0.360  0.370  否
5   乌黑  蜷缩  沉闷  清晰  凹陷  硬滑  0.774  0.376  是
6   浅白  蜷缩  浊响  模糊  平坦  软粘  0.343  0.099  否
7   乌黑  蜷缩  浊响  清晰  凹陷  硬滑  0.634  0.264  是
8   浅白  稍蜷  沉闷  稍糊  凹陷  硬滑  0.657  0.198  否
9   青绿  稍蜷  浊响  清晰  稍凹  软粘  0.403  0.237  是
10  青绿  蜷缩  浊响  清晰  凹陷  硬滑  0.697  0.460  是
11  青绿  稍蜷  浊响  稍糊  凹陷  硬滑  0.639  0.161  否
12  浅白  蜷缩  浊响  模糊  平坦  硬滑  0.593  0.042  否
13  青绿  蜷缩  沉闷  清晰  凹陷  硬滑  0.608  0.318  是
14  青绿  硬挺  清脆  清晰  平坦  软粘  0.243  0.267  否
15  乌黑  稍蜷  沉闷  稍糊  稍凹  硬滑  0.666  0.091  否
16  浅白  硬挺  清脆  模糊  平坦  硬滑  0.245  0.057  否
17 11 6


In [3]:
def compute_Inf_Entropy(data_table):
    '''
    compute information entropy, 
    the input is the data table and the output is the information entropy
    '''
    num_class1 = len(data_table.loc[data_table['好瓜']=='是'])
    num_class2 = len(data_table.loc[data_table['好瓜']=='否'])
    num_all = len(data_table)
    if num_class1==0 or num_class2==0:
        return 0
    Inf_Entropy = -(num_class1/num_all * log(num_class1/num_all, 2)+ num_class2/num_all * log(num_class2/num_all, 2))
    return Inf_Entropy

def Inf_gain(data_table, column):
    '''
    compute information gain, 
    the input is the data table and the feature that we want to compute the information gain.
    the output is the information gain
    '''
    raw_inf_entropy = compute_Inf_Entropy(data_table)
    
    for element in raw_dataset[column].unique():
        subset = data_table.loc[data_table[column]==element]
        raw_inf_entropy = raw_inf_entropy - len(subset)/len(data_table) * compute_Inf_Entropy(subset)
    return raw_inf_entropy

def find_best_column(data_table):
    '''
    compare all the features and find the one with max information gain 
    '''
    max_gain = 0
    best_col = None
    for col in data_table.iloc[:,:-3].columns:
        col_Inf_gain = Inf_gain(data_table,col)
        if max_gain < col_Inf_gain:
            max_gain = col_Inf_gain
            best_col = col
    return best_col

def create_tree(data_table,raw_dataset):
    '''
    Creating Decision Trees with Recursion
    '''
    if len(data_table['好瓜'].unique())==1: # Same category
        return data_table['好瓜'].unique()[0]
    
    if len(data_table.iloc[:,:-3].columns)==0: # iterating through all features
        num_good = len(data_table.loc[data_table['好瓜']=='是'])
        num_bad = len(data_table.loc[data_table['好瓜']=='否'])
        return '是' if num_good>num_bad else '否'
    
    best_feature = find_best_column(data_table)

    tree = {best_feature:{}}
    # print(raw_dataset[best_feature].unique())
    for feature_value in raw_dataset[best_feature].unique():
        subset = data_table.loc[data_table[best_feature]==feature_value].copy()
        subset.pop(best_feature) # delete the feature
        
        if len(subset)==0:
            num_good = len(data_table.loc[data_table['好瓜']=='是'])
            num_bad = len(data_table.loc[data_table['好瓜']=='否'])
            tree[best_feature][feature_value] = '是' if num_good>num_bad else '否'
            continue
        tree[best_feature][feature_value]=create_tree(subset,raw_dataset)
    return tree

def test_DTree(data_table,tree):
    '''
    Test the decision tree model on the test set
    '''
    def test_one_sample(data_vec,tree):
        if tree == '是' or tree == '否':
            return tree
        feature = list(tree.keys())[0]
        
        feature_value = list(tree[feature].keys())

        subtree = tree[feature][data_vec[feature]]
        return test_one_sample(data_vec,subtree)
    
    
    correct_prediction = 0
    wrong_prediction = 0
    for index, sample in data_table.iterrows():
        pred_label = test_one_sample(sample,tree)
        if pred_label == sample['好瓜']:
            correct_prediction = correct_prediction+1
        else :
            wrong_prediction = wrong_prediction+1
    return correct_prediction/(correct_prediction+wrong_prediction)

In [4]:
tree = create_tree(train_set,raw_dataset)
print("the DTree is:")
print(tree)
acc = test_DTree(test_set,tree)
print('the accuracy on the test set is: '+ str(acc))

the DTree is:
{'纹理': {'稍糊': {'色泽': {'青绿': '否', '乌黑': '是', '浅白': '否'}}, '清晰': {'触感': {'硬滑': '是', '软粘': {'色泽': {'青绿': '是', '乌黑': '否', '浅白': '否'}}}}, '模糊': '否'}}
the accuracy on the test set is: 0.6666666666666666
